In [ ]:
import pandas as pd
import json
import datetime
import time
import os
import math
import numpy as np
from config import DATA_CONSUMPTION_PROCESSED_FILE, DATA_VACATIONS_FILE, DATA_METADATA_PROCESSED_FILE, DATA_VACATIONS_INTERVENTION_FILE
import matplotlib
from sklearn import mixture

In [ ]:
quantile_lower = 0.10
quantile_upper = 0.90

In [ ]:
real_consumption_df = pd.read_csv(DATA_CONSUMPTION_PROCESSED_FILE)
real_consumption_df['timestamp'] = pd.to_datetime(real_consumption_df['timestamp'])
test = real_consumption_df.drop_duplicates(['timestamp','smapee'])
test.reset_index(inplace=True)
test = real_consumption_df.drop(['timestamp','INTERVENTION'], axis=1)
minimum = test.groupby('smapee').quantile(quantile_lower)
min_max = minimum.merge(test.groupby('smapee').quantile(quantile_upper), left_index=True, right_index=True, suffixes=("min","max"))

In [ ]:
result = real_consumption_df.merge(min_max, left_on='smapee', left_index=False, right_index=True)
result['Full Occupancy'] = result.apply(lambda x: -1 if x['CONSUMPTION_kWh']<=x['CONSUMPTION_kWhmin'] else 0, axis=1)
result['Null Occupancy'] = result.apply(lambda x: 1 if x['CONSUMPTION_kWh']>=x['CONSUMPTION_kWhmax'] else 0, axis=1)
# result = add_sg(result, group='smapee')
result

In [ ]:
graph = result[result['smapee']=='ID28763']
graph = graph.drop_duplicates('timestamp')
graph = graph.sort_values(by=['timestamp'])
graph = graph.reset_index(drop=True)
graph['timestamp'] = pd.to_datetime(graph['timestamp'].values)
graph = graph.set_index('timestamp')

In [ ]:
graph['Daily Consumption'] = graph['CONSUMPTION_kWh']
visual = graph[['Daily Consumption']]
font = {'family' : 'Arial',
        'size'   : 18}
ax = visual.plot()
ax.set_xlabel("")
ax.legend(['Daily Consumption [kWh]']);
matplotlib.rc('font', **font)
plt.rcParams["figure.figsize"] = (6,3)

In [ ]:
visual = graph[['Full Occupancy', 'Null Occupancy']]
visual['Full Occupancy'] = -visual['Full Occupancy'] #just for graphical properties
font = {'family' : 'Arial',
        'size'   : 18}
ax = visual.plot()
ax.set_xlabel("")
ax.set_ylabel("Boolean [-]")
ax.legend(['Full Occupancy', 'Null Occupancy']);
matplotlib.rc('font', **font)
idx = ["","0","","1"]
ax.set_yticklabels(idx)
plt.rcParams["figure.figsize"] = (6,3)

In [ ]:
result[['smapee','timestamp', 'EMPTY_DAY', 'FULL_DAY', 'INTERVENTION']].to_csv(DATA_VACATIONS_FILE, index=False)

# per intervention

In [ ]:
quantile_lower = 0.10
quantile_upper = 0.90
group = real_consumption_df.groupby(['timestamp','INTERVENTION'])[['CONSUMPTION_kWh']].mean()
group = group.reset_index()
test = group.drop(['timestamp'], axis=1)
minimum = test.groupby('INTERVENTION').quantile(quantile_lower)
min_max = minimum.merge(test.groupby('INTERVENTION').quantile(quantile_upper), left_index=True, right_index=True, suffixes=("min","max"))
result_group = group.merge(min_max, left_on='INTERVENTION', left_index=False, right_index=True)
result_group['EMPTY_DAY_INT_'+str(quantile_lower)] = result_group.apply(lambda x: 1 if x['CONSUMPTION_kWh']<=x['CONSUMPTION_kWhmin'] else 0, axis=1)
result_group['FULL_DAY_INT_'+str(quantile_upper)] = result_group.apply(lambda x: 1 if x['CONSUMPTION_kWh']>=x['CONSUMPTION_kWhmax'] else 0, axis=1)
result_group = result_group.set_index('timestamp')    
    
quantile_lower = 0.25
quantile_upper = 0.75
group = real_consumption_df.groupby(['timestamp','INTERVENTION'])[['CONSUMPTION_kWh']].mean()
group = group.reset_index()
test = group.drop(['timestamp'], axis=1)
minimum = test.groupby('INTERVENTION').quantile(quantile_lower)
min_max = minimum.merge(test.groupby('INTERVENTION').quantile(quantile_upper), left_index=True, right_index=True, suffixes=("min","max"))

result_group2 = group.merge(min_max, left_on='INTERVENTION', left_index=False, right_index=True)
result_group2['EMPTY_DAY_INT_'+str(quantile_lower)] = result_group2.apply(lambda x: 1 if x['CONSUMPTION_kWh']<=x['CONSUMPTION_kWhmin'] else 0, axis=1)
result_group2['FULL_DAY_INT_'+str(quantile_upper)] = result_group2.apply(lambda x: 1 if x['CONSUMPTION_kWh']>=x['CONSUMPTION_kWhmax'] else 0, axis=1)
result_group2 = result_group2.set_index('timestamp')    

result_group = result_group.merge(result_group2, on=['timestamp','INTERVENTION'])

In [ ]:
result_group[['EMPTY_DAY_INT_0.1','FULL_DAY_INT_0.9', 'EMPTY_DAY_INT_0.25','FULL_DAY_INT_0.75', 'INTERVENTION']].to_csv(DATA_VACATIONS_INTERVENTION_FILE, index=True)

In [ ]:
result_group